In [ ]:
import pandas as pd
import os
dhs_wca = r'/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp3/data/dhs/dhs_wca/'
df = pd.read_stata(os.path.join(dhs_wca, 'AO_2015-16_DHS_09032024_853_207844', 'AOIR71DT','AOIR71FL.DTA'), convert_categoricals=False)

In [2]:
df['exposed_preg'] = ((df['v213'] > 0) & (df['v201'] > 0)).astype(int)

/tmp/ipykernel_1200367/3720273993.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['exposed_preg'] = ((df['v213'] > 0) & (df['v201'] > 0)).astype(int)


In [3]:
df = df[df['v013'] == 1] ## age group 15-19
def make_string(integer, length = 8):
    return str(integer).zfill(length)
df['id'] = 'AO'+'2015'+ df['v004'].apply(make_string)
df = df.groupby('id').mean(numeric_only=True)

/tmp/ipykernel_1200367/892424802.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['id'] = 'AO'+'2015'+ df['v004'].apply(make_string)


In [7]:
df.shape

(614, 5326)

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import json
from tqdm import tqdm
def get_youth_preg(source_path):
   
    for f in os.listdir(source_path):
        if 'IR' in f:
            individual_datafile = os.path.join(source_path, f, find_sub_file(source_path+f,'dta'))
    survey_year = source_path.split('/')[-2][3:7]
    country_code = source_path.split('/')[-2][:2]

    dhs_ir = pd.read_stata(individual_datafile, convert_categoricals=False)
    dhs_ir['id'] = country_code[:2]+survey_year+ dhs_ir['v004'].apply(make_string)

    youth_df = dhs_ir[dhs_ir['v013'] == 1] ## age group 15-19
    youth_df['exposed_preg'] = ((youth_df['v213'] > 0) & (youth_df['v201'] > 0)).astype(int)
    youth_df = youth_df.groupby('id').mean(numeric_only=True).reset_index()
    return youth_df

with open(f'../survey_processing/dhs_country_code.json', 'r') as file:
    dhs_cc = json.load(file)
root_grid = dhs_wca
youth_dfs = []
print('Summarizing youth pregnancy...')
for f in tqdm(os.listdir(root_grid)):
    if 'DHS' in f:
        try:
            youth_df = get_youth_preg(root_grid+f+'/')
            
            youth_dfs.append(youth_df)
        except Exception as e:
            print(e)
            


dhs_df_all = pd.concat(youth_dfs)

## Calculating Centroids
print('Calculating geospatial variables...')
gdfs = []
for f in tqdm(os.listdir(root_grid)):
    if 'DHS' in f:
        for sub_f in os.listdir(os.path.join(root_grid,f)):
            if sub_f.__contains__('GE'):
                shape_file = os.path.join(root_grid, f, sub_f)
                gdf = gpd.read_file(shape_file)
                # Append to the list of GeoDataFrames
                gdfs.append(gdf)
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
def country_code_to_name(country_code):
    return dhs_cc[country_code]
print('Generating ...')
combined_gdf['COUNTRY'] = combined_gdf['DHSCC'].apply(country_code_to_name)
combined_gdf['SURVEY_NAME'] = [combined_gdf.iloc[i]['COUNTRY']+'_DHS_'+str(int(combined_gdf.iloc[i]['DHSYEAR'])) for i in range(combined_gdf.shape[0])]
combined_gdf['YEAR'] =combined_gdf['DHSYEAR'].apply(int)
combined_gdf['CENTROID_ID']  = combined_gdf['DHSID']

centroid_df = combined_gdf[['CENTROID_ID', 'SURVEY_NAME', 'COUNTRY','YEAR', 'LATNUM', 'LONGNUM']]
centroid_df = centroid_df[~((centroid_df['LATNUM'] == 0) & (centroid_df['LONGNUM'] == 0))]
centroid_df.drop_duplicates(inplace=True)
dhs_df_all.drop_duplicates(inplace=True)
centroid_df = centroid_df.reset_index()

merged_centroid_df = pd.merge(centroid_df, dhs_df_all, left_on='CENTROID_ID', right_on='id', how='left')
print('Saving data...')

save_split(merged_centroid_df)